Домашку будет легче делать в колабе (убедитесь, что у вас runtype с gpu).

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна в большую или меньшую сторону
3) измените размерность итоговых векторов

Выберете несколько не похожих по смыслу слов (не таких как в семинаре), и протестируйте полученные эмбединги (найдите ближайшие слова и оцените качество, как в семинаре). 
Постарайтесь обучать модели как можно дольше и на как можно большем количестве данных. (Но если у вас мало времени или ресурсов, то допустимо взять поменьше данных и поставить меньше эпох)

In [14]:
import os
import re
import requests
import zipfile
from tqdm import tqdm

import pandas as pd
import numpy as np
from collections import Counter

import nltk
from nltk.tokenize import word_tokenize
import pymorphy2
nltk.download('punkt')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import gensim

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from itertools import chain

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Используемое устройство: mps


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/akatsnelson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
url = "https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.1/lenta-ru-news.csv.bz2"
file_name = "lenta-ru-news.csv.bz2"
if not os.path.exists(file_name):
    with open(file_name, "wb") as f:
        r = requests.get(url)
        f.write(r.content)


In [3]:
lenta_df = pd.read_csv(file_name, compression='bz2')

morph = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    if not isinstance(text, str):
        return []
    text = text.lower()
    tokens = re.findall('[а-яё]+', text)
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    lemmas = [lemma for lemma in lemmas if len(lemma) > 2]
    return lemmas

lenta_df_sample = lenta_df.sample(500, random_state=42) # долго джать больше извните(
texts = [preprocess_text(text) for text in tqdm(lenta_df_sample['text'])]
texts = [text for text in texts if text]
print(f"Обработано {len(texts)} текстов.")

/var/folders/pm/xshykg4d7wvgy969hfm8hgp40000gn/T/ipykernel_11219/2111616842.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  lenta_df = pd.read_csv(file_name, compression='bz2')
100%|█████████████████████████████████████████| 500/500 [00:07<00:00, 68.40it/s]

Обработано 499 текстов.


In [18]:
word_counts = Counter(chain.from_iterable(texts))
vocab = [word for word, count in word_counts.items() if count >= 5]

word_to_ind = {word: i for i, word in enumerate(vocab)}
ind_to_word = {i: w for w, i in word_to_ind.items()}

indexed_texts = [
    [word_to_ind[word] for word in text if word in word_to_ind]
    for text in texts
]
indexed_texts = [text for text in indexed_texts if len(text) > 1]

word_freqs = np.array([word_counts[word] for word in vocab])
word_freqs_powered = word_freqs ** 0.75
unigram_dist = torch.from_numpy(word_freqs_powered / word_freqs_powered.sum()).float()

def get_training_data(indexed_texts, window_size, num_negative_samples):
    center_words = []
    context_words = []
    negative_words = []

    unigram_probs = unigram_dist

    for text in tqdm(indexed_texts, desc="Генерация обучающих пар"):
        text_len = len(text)
        for i in range(text_len):
            window_start = max(0, i - window_size)
            window_end = min(text_len, i + window_size + 1)
            center = text[i]

            for j in range(window_start, window_end):
                if i == j:
                    continue
                context = text[j]

                center_words.append(center)
                context_words.append(context)

                neg_samples = torch.multinomial(unigram_probs, num_negative_samples, replacement=True)
                negative_words.append(neg_samples)

    return (torch.tensor(center_words, dtype=torch.long),
            torch.tensor(context_words, dtype=torch.long),
            torch.stack(negative_words))

WINDOW_SIZE = 3
NUM_NEGATIVE_SAMPLES = 5

center_words, pos_context, neg_context = get_training_data(indexed_texts, WINDOW_SIZE, NUM_NEGATIVE_SAMPLES)

Генерация обучающих пар: 100%|███████████████| 499/499 [00:03<00:00, 127.15it/s]


In [19]:
class Word2VecDataset(Dataset):
    def __init__(self, center, pos, neg):
        self.center = center
        self.pos = pos
        self.neg = neg

    def __len__(self):
        return len(self.center)

    def __getitem__(self, idx):
        return self.center[idx], self.pos[idx], self.neg[idx]

class SkipGramNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.in_embed = nn.Embedding(vocab_size, embedding_dim)
        self.out_embed = nn.Embedding(vocab_size, embedding_dim)
        
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)

    def forward(self, center_words, context_words, negative_words):
        center_emb = self.in_embed(center_words) 
        context_emb = self.out_embed(context_words) 
        negative_emb = self.out_embed(negative_words)

        log_pos = torch.sum(torch.mul(center_emb, context_emb), dim=1)
        neg_scores = torch.bmm(negative_emb, center_emb.unsqueeze(2)).squeeze(2)

        log_pos = F.logsigmoid(log_pos)
        log_neg = F.logsigmoid(-neg_scores).sum(1)
        
        loss = -(log_pos + log_neg).mean()
        return loss

In [20]:
VOCAB_SIZE = len(vocab)
BATCH_SIZE = 1024
EPOCHS = 5 
LEARNING_RATE = 0.001
EMBEDDING_DIM = 100

skipgram_model = SkipGramNegativeSampling(VOCAB_SIZE, EMBEDDING_DIM).to(device)
dataset = Word2VecDataset(center_words, pos_context, neg_context)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
optimizer = optim.Adam(skipgram_model.parameters(), lr=LEARNING_RATE)

for epoch in range(EPOCHS):
    total_loss = 0
    for i, (center, pos, neg) in enumerate(tqdm(dataloader, desc=f"Эпоха {epoch+1}/{EPOCHS}")):
        center, pos, neg = center.to(device), pos.to(device), neg.to(device)

        optimizer.zero_grad()
        loss = skipgram_model(center, pos, neg)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Эпоха {epoch+1}, Средняя ошибка: {total_loss / len(dataloader):.4f}")

skipgram_embeddings = skipgram_model.in_embed.weight.data.cpu().numpy()

Эпоха 1/5: 100%|█████████████████████████████| 328/328 [00:01<00:00, 173.96it/s]


Эпоха 1, Средняя ошибка: 8.4296


Эпоха 2/5: 100%|█████████████████████████████| 328/328 [00:01<00:00, 170.56it/s]


Эпоха 2, Средняя ошибка: 7.2239


Эпоха 3/5: 100%|█████████████████████████████| 328/328 [00:02<00:00, 161.06it/s]


Эпоха 3, Средняя ошибка: 6.1883


Эпоха 4/5: 100%|█████████████████████████████| 328/328 [00:01<00:00, 170.00it/s]


Эпоха 4, Средняя ошибка: 5.1885


Эпоха 5/5: 100%|█████████████████████████████| 328/328 [00:01<00:00, 181.78it/s]

Эпоха 5, Средняя ошибка: 4.3069


In [21]:
def find_nearest(word, embeddings, top_n=10):
    if word not in word_to_ind:
        print(f"Слово '{word}' не найдено в словаре.")
        return

    word_idx = word_to_ind[word]
    word_vec = embeddings[word_idx]

    similarities = cosine_similarity([word_vec], embeddings)[0]

    nearest_indices = similarities.argsort()[-top_n-1:-1][::-1]

    print(f"Ближайшие слова для '{word}':")
    for idx in nearest_indices:
        print(f"  - {ind_to_word[idx]} (сходство: {similarities[idx]:.4f})")

test_words = ['россия', 'нефть', 'любовь', 'компьютер']
for word in test_words:
    find_nearest(word, skipgram_embeddings)
    print("-" * 20)

Ближайшие слова для 'россия':
  - командир (сходство: 0.4272)
  - поставка (сходство: 0.4255)
  - процент (сходство: 0.4150)
  - решение (сходство: 0.4036)
  - они (сходство: 0.3939)
  - счёт (сходство: 0.3819)
  - подписать (сходство: 0.3685)
  - центральный (сходство: 0.3650)
  - кпрф (сходство: 0.3638)
  - момент (сходство: 0.3637)
--------------------
Ближайшие слова для 'нефть':
  - инвестиционный (сходство: 0.3618)
  - секретарь (сходство: 0.3617)
  - жилец (сходство: 0.3585)
  - матч (сходство: 0.3550)
  - эффективность (сходство: 0.3485)
  - принятие (сходство: 0.3469)
  - ленинградский (сходство: 0.3345)
  - доступно (сходство: 0.3336)
  - медведев (сходство: 0.3159)
  - министр (сходство: 0.3125)
--------------------
Слово 'любовь' не найдено в словаре.
--------------------
Ближайшие слова для 'компьютер':
  - специалист (сходство: 0.4346)
  - пресс (сходство: 0.3612)
  - коррупция (сходство: 0.3498)
  - предыдущий (сходство: 0.3456)
  - парламентский (сходство: 0.3399)
  - в

# Задание 2 (2 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [22]:
from gensim.models import Word2Vec, FastText

w2v_model_gensim = Word2Vec(
    sentences=texts,    
    vector_size=150,
    window=8,
    min_count=10,
    sg=1,
    hs=0,
    negative=15,
    epochs=10,
    workers=4 
)

In [23]:
ft_model_gensim = FastText(
    sentences=texts,
    vector_size=200,
    window=5,
    min_count=8,
    sg=1,
    epochs=10,
    min_n=3,
    max_n=6,
    workers=4
)
ft_model_gensim.max_n = 5

In [24]:
for word in test_words:
    if word in w2v_model_gensim.wv:
        print(f"Ближайшие к '{word}': {w2v_model_gensim.wv.most_similar(word, topn=5)}")
    else:
        print(f"Слово '{word}' не найдено в модели Word2Vec.")

for word in test_words:
    if word in ft_model_gensim.wv:
        print(f"Ближайшие к '{word}': {ft_model_gensim.wv.most_similar(word, topn=5)}")
    else:
        print(f"Слово '{word}' не найдено в словаре FastText, но вектор можно получить.")

Ближайшие к 'россия': [('крым', 0.8023478388786316), ('корея', 0.7896313071250916), ('путин', 0.7829669117927551), ('владимир', 0.7795822620391846), ('казахстан', 0.7745606303215027)]
Ближайшие к 'нефть': [('нефтяной', 0.8904008865356445), ('добыча', 0.8843613862991333), ('поставка', 0.8790572881698608), ('завод', 0.8719830513000488), ('газпром', 0.8709684014320374)]
Слово 'любовь' не найдено в модели Word2Vec.
Ближайшие к 'компьютер': [('телефон', 0.9519345760345459), ('смартфон', 0.9435363411903381), ('пользователь', 0.9263903498649597), ('способный', 0.9257462024688721), ('диск', 0.9249487519264221)]
Ближайшие к 'россия': [('кандидат', 0.9551265835762024), ('партия', 0.9515881538391113), ('миссия', 0.9464306831359863), ('белоруссия', 0.9422648549079895), ('российский', 0.9307098984718323)]
Ближайшие к 'нефть': [('роснефть', 0.9827443361282349), ('долг', 0.979930579662323), ('доля', 0.9720900654792786), ('доллар', 0.9602851271629333), ('нефтяной', 0.9593896865844727)]
Ближайшие к 'лю

# Задание 3 (3 балла)

Используя датасет для классификации (labeled.csv), обучите классификатор на базе эмбеддингов. Оцените качество на отложенной выборке.   
В качестве эмбеддинг модели вы можете использовать одну из моделей обученных в предыдущем задании или использовать одну из предобученных моделей с rusvectores (удостоверьтесь что правильно воспроизводите предобработку в этом случае!)  
Для того, чтобы построить эмбединг целого текста, усредните вектора отдельных слов в один общий вектор. 
В качестве алгоритма классификации используйте LogisicticRegression (можете попробовать SGDClassifier, чтобы было побыстрее)  
F1 мера должна быть выше 20%. 

In [25]:
url_labeled = "https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/notebooks/word_embeddings/labeled.csv"
file_labeled = "labeled.csv"
if not os.path.exists(file_labeled):
    r = requests.get(url_labeled)
    with open(file_labeled, 'wb') as f:
        f.write(r.content)

In [26]:
df_class = pd.read_csv(file_labeled)
df_class['toxic'] = (df_class['toxic'] > 0).astype(int)

df_class['lemmas'] = df_class['comment'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(
    df_class['lemmas'],
    df_class['toxic'],
    test_size=0.2,
    random_state=42,
    stratify=df_class['toxic']
)

In [27]:
def text_to_vector(text_lemmas, model):
    word_vectors = []
    for word in text_lemmas:
        if word in model.wv:
            word_vectors.append(model.wv[word])

    if not word_vectors:
        return np.zeros(model.vector_size)

    return np.mean(word_vectors, axis=0)

embedding_model = ft_model_gensim

print("Векторизация текстов")
X_train_vec = np.array([text_to_vector(text, embedding_model) for text in tqdm(X_train)])
X_test_vec = np.array([text_to_vector(text, embedding_model) for text in tqdm(X_test)])

Векторизация текстов


100%|█████████████████████████████████████| 2883/2883 [00:00<00:00, 7032.41it/s]


In [28]:
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_vec, y_train)

y_pred = log_reg.predict(X_test_vec)

f1 = f1_score(y_test, y_pred, average='weighted')
print(f"\nF1-мера (weighted) на тестовой выборке: {f1:.4f}")

if f1 > 0.20:
    print("Успех! F1-мера выше 20%.")
else:
    print("F1-мера ниже 20%.")

print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred))


F1-мера (weighted) на тестовой выборке: 0.7418
Успех! F1-мера выше 20%.

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.77      0.91      0.83      1918
           1       0.72      0.46      0.56       965

    accuracy                           0.76      2883
   macro avg       0.74      0.68      0.70      2883
weighted avg       0.75      0.76      0.74      2883



# Задание 4 (2 доп балла)

В тетрадку с фастекстом добавьте код для обучения с negative sampling (задача сводится к бинарной классификации) и обучите модель. Проверьте полученную модель на нескольких словах. Похожие слова должны быть похожими по смыслу и по форме.

In [29]:
def get_subwords(word, min_n=3, max_n=6):
    """Создает n-граммы для слова."""
    subwords = []
    word = '<' + word + '>'
    for i in range(len(word)):
        for j in range(i + min_n, min(i + max_n + 1, len(word) + 1)):
            subwords.append(word[i:j])
    return subwords

subword_vocab = set()
for word in vocab:
    subwords = get_subwords(word, min_n=3, max_n=6)
    for sub in subwords:
        subword_vocab.add(sub)


full_vocab = vocab + list(subword_vocab)
print(f"Размер основного словаря: {len(vocab)}")
print(f"Размер полного словаря (с n-граммами): {len(full_vocab)}")


word_to_ind_ft = {word: i for i, word in enumerate(full_vocab)}
ind_to_word_ft = {i: word for i, word in enumerate(full_vocab)}

Размер основного словаря: 2744
Размер полного словаря (с n-граммами): 32695


In [33]:
class FastTextNegativeSampling(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super().__init__()
        self.in_embed = nn.Embedding(vocab_size, embedding_dim)
        self.out_embed = nn.Embedding(len(vocab), embedding_dim)
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)

    def get_word_vector(self, word_idx_with_subwords):
        word_emb = self.in_embed(word_idx_with_subwords)
        return torch.mean(word_emb, dim=0)

    def forward(self, center_indices, context_indices, negative_indices):
        center_emb_list = []
        for word_subwords in center_indices:
            word_subwords_tensor = torch.LongTensor(word_subwords).to(device)
            center_emb_list.append(self.get_word_vector(word_subwords_tensor))

        center_emb = torch.stack(center_emb_list) 

        context_emb = self.out_embed(context_indices)
        negative_emb = self.out_embed(negative_indices)

        log_pos = torch.sum(torch.mul(center_emb, context_emb), dim=1)
        log_pos = F.logsigmoid(log_pos)

        neg_scores = torch.bmm(negative_emb, center_emb.unsqueeze(2)).squeeze(2)
        log_neg = F.logsigmoid(-neg_scores).sum(1)

        loss = -(log_pos + log_neg).mean()
        return loss

def get_indices_with_subwords(word_idx):
    word = ind_to_word[word_idx]
    subwords = get_subwords(word)
    indices = [word_to_ind_ft.get(w, -1) for w in [word] + subwords]
    return [i for i in indices if i != -1]

In [39]:
ft_model_scratch = FastTextNegativeSampling(len(full_vocab), EMBEDDING_DIM).to(device)
optimizer_ft = optim.Adam(ft_model_scratch.parameters(), lr=0.001)

num_samples_to_train = 10000
train_subset_indices = np.random.choice(len(center_words), num_samples_to_train)

ft_model_scratch.train()
for i in tqdm(range(num_samples_to_train)):
    idx = train_subset_indices[i]
    center_idx = center_words[idx].item()
    context_idx = pos_context[idx].unsqueeze(0).to(device) # batch_size=1
    neg_idx = neg_context[idx].unsqueeze(0).to(device) # batch_size=1


    center_with_subwords_indices = [get_indices_with_subwords(center_idx)]

    optimizer_ft.zero_grad()
    loss = ft_model_scratch(center_with_subwords_indices, context_idx, neg_idx)
    loss.backward()
    optimizer_ft.step()


100%|████████████████████████████████████| 10000/10000 [00:42<00:00, 237.75it/s]


In [40]:
def get_ft_vector_scratch(word, model):
    subwords = get_subwords(word)
    indices = [word_to_ind_ft.get(w, -1) for w in [word] + subwords]
    indices = [i for i in indices if i != -1]
    if not indices:
        return np.zeros(model.in_embed.embedding_dim)

    with torch.no_grad():
        indices_tensor = torch.LongTensor(indices).to(device)
        vector = model.in_embed(indices_tensor).mean(dim=0).cpu().numpy()
    return vector

def find_nearest_ft_scratch(word, model, top_n=10):
    word_vec = get_ft_vector_scratch(word, model)

    all_word_vectors = []
    for i in range(len(vocab)):
        v = get_ft_vector_scratch(ind_to_word[i], model)
        all_word_vectors.append(v)
    all_word_vectors = np.array(all_word_vectors)

    similarities = cosine_similarity([word_vec], all_word_vectors)[0]
    nearest_indices = similarities.argsort()[-top_n-1:-1][::-1]

    print(f"Ближайшие слова для '{word}' (FastText с нуля):")
    for idx in nearest_indices:
        print(f"  - {ind_to_word[idx]} (сходство: {similarities[idx]:.4f})")

In [41]:
ft_model_scratch.eval()

find_nearest_ft_scratch('учитель', ft_model_scratch)
print("-" * 20)
find_nearest_ft_scratch('крым', ft_model_scratch)

Ближайшие слова для 'учитель' (FastText с нуля):
  - житель (сходство: 0.4780)
  - руководитель (сходство: 0.4626)
  - водитель (сходство: 0.4352)
  - представитель (сходство: 0.4197)
  - строитель (сходство: 0.4153)
  - посетитель (сходство: 0.4141)
  - исполнитель (сходство: 0.3998)
  - строительство (сходство: 0.3966)
  - значительный (сходство: 0.3896)
  - заместитель (сходство: 0.3815)
--------------------
Ближайшие слова для 'крым' (FastText с нуля):
  - приём (сходство: 0.3588)
  - миссия (сходство: 0.3259)
  - выбор (сходство: 0.3188)
  - донбасс (сходство: 0.3160)
  - ребёнок (сходство: 0.2883)
  - минимум (сходство: 0.2684)
  - уволить (сходство: 0.2663)
  - благодаря (сходство: 0.2659)
  - форма (сходство: 0.2543)
  - информация (сходство: 0.2468)
